# Week2_1 Assignment

# [BASIC](#Basic)
- BERT 모델의 hidden state에서 **특정 단어의 embedding을 여러 방식으로 추출 및 생성**할 수 있다.

# [CHALLENGE](#Challenge)
- **cosine similarity 함수를 구현**할 수 있다. 
- **단어들의 유사도**를 cosine similarity로 비교할 수 있다. 

# [ADVANCED](#Advanced)
- 문장 embedding을 구해 **문장 간 유사도**를 구할 수 있다.

### Reference
- [BERT word embedding & sentence embedding tutorial 영문 블로그](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/#33-creating-word-and-sentence-vectors-from-hidden-states)

In [ ]:
import os
import sys
import pandas as pd
import numpy as np 
import torch
import random

## Basic

### BERT 모델과 토크나이저 로드   
- 두 사람의 대화에서 (단어 및 문장의) embedding을 생성하고자 한다. 아래 대화를 BERT 모델에 입력해 출력값 중 "hidden states"값을 가져오자.
- `Hidden States`는 3차원 텐서를 가지고 있는 list 타입이다. List에는 BERT 모델의 각 layer마다의 hidden state 3차원 텐서를 갖고 있으며 각 텐서는 (batch_size, sequence_length, hidden_size) shape을 가진다. BERT-base 모델은 12 layer를 갖고 있고 이와 별도로 Embedding Layer 1개를 더 갖고 있기 때문에 `len(hidden states)`는 13개가 된다. 
    - batch_size: 학습 시 설정한 배치 사이즈. 또는 BERT 모델에 입력된 문장의 개수
    - sequence_length: 문장의 token의 개수. 
    - hidden size: token의 embedding size 
- Reference
    - [BertTokenizer.tokenize() 함수의 매개변수 설명](https://huggingface.co/transformers/v3.0.2/main_classes/tokenizer.html#transformers.PreTrainedTokenizer.__call__)
    - [BERTModel.forward() 함수의 매개변수 및 리턴 값 설명](https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel.forward)

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertModel

In [ ]:
tokenizer_bert = BertTokenizer.from_pretrained("bert-base-cased")
model_bert = BertModel.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
normal_person = ["what do you do when you have free time?"]
nerd = ["I code. code frees my minds, body and soul."]
normal_person.append("(what a nerd...) coding?")
nerd.append("Yes. coding is the best thing to do in the free time.")

for i in range(len(normal_person)):
    print(f"Normal Person asked: {normal_person[i]}")
    print(f"Nerd answers: {nerd[i]}")

Normal Person asked: what do you do when you have free time?
Nerd answers: I code. code frees my minds, body and soul.
Normal Person asked: (what a nerd...) coding?
Nerd answers: Yes. coding is the best thing to do in the free time.


In [ ]:
# 매개변수 설명
# truncation <- max_len 넘어가지 않도록 자르기
# padding <- max(seq_len, max_len) zero padding
# return_tensors <- return 2d tensor 

inputs = tokenizer_bert(
    text = normal_person,
    text_pair = nerd,
    truncation = True,
    padding = "longest",
    return_tensors='pt'
    )

print(inputs['input_ids'].shape)

torch.Size([2, 28])


In [ ]:
inputs

{'input_ids': tensor([[  101,  1184,  1202,  1128,  1202,  1165,  1128,  1138,  1714,  1159,
           136,   102,   146,  3463,   119,  3463,  1714,  1116,  1139, 10089,
           117,  1404,  1105,  3960,   119,   102,     0,     0],
        [  101,   113,  1184,   170, 24928,  2956,   119,   119,   119,   114,
         19350,   136,   102,  2160,   119, 19350,  1110,  1103,  1436,  1645,
          1106,  1202,  1107,  1103,  1714,  1159,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1]])}

In [ ]:
# decoding
for i in range(len(inputs['input_ids'])):
    print(f"Coversation {i} -> '{tokenizer_bert.decode(inputs['input_ids'][i])}'")

Coversation 0 -> '[CLS] what do you do when you have free time? [SEP] I code. code frees my minds, body and soul. [SEP] [PAD] [PAD]'
Coversation 1 -> '[CLS] ( what a nerd... ) coding? [SEP] Yes. coding is the best thing to do in the free time. [SEP]'


In [ ]:
# "code" 단어의 token id(각 단어에게 고유하게 주어진 id)를 출력
tokenizer_bert.encode('code', add_special_tokens=False)

[3463]

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


In [ ]:
# 입력 데이터와 BERT 모델을 "GPU" 장치로 로드함
inputs = inputs.to(device)
model_bert.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# 입력 데이터를 BERT 모델에 넣어 출력값을 가져옴
outputs = model_bert(
    **inputs, 
    output_hidden_states=True
    )

In [ ]:
hidden_states = outputs['hidden_states']
print(f"# layers : {len(hidden_states)}")
print(f"tensor shape in each layer : {hidden_states[-1].shape}")

# layers : 13
tensor shape in each layer : torch.Size([2, 28, 768])


###  Q1. 1번째 sequence (문장)에서 "code"라는 단어의 인덱스를 모두 반환하라.
- "code" 단어는 총 2개 존재 

In [ ]:
def get_index(seq, word):
    encoded_tok = tokenizer_bert.encode(word, add_special_tokens=False)[0]
    index = (seq == encoded_tok).nonzero(as_tuple=False)
    return index

# input
# seq1: 1번째 sequence
# token: 단어
seq1 = inputs['input_ids'][0]
token = "code"

# output
token_index = get_index(seq1, token)
print(token_index)

tensor([[13],
        [15]], device='cuda:0')


### Q2. 1번째 sequence의 1번째 "code" 토큰의 embedding을 여러가지 방식으로 구하고자 한다. BERT hidden state를 다음의 방식으로 인덱싱해 embedding을 구하라
- 1 layer
- last layer
- sum all 12 layers
- sum last 4 layers
- concat last 4 layers
- average last 4 layers

In [ ]:
torch.concat([outputs['hidden_states'][_][0][token_index[0]] for _ in range(len(outputs['hidden_states'][-4:]))], axis=1)

tensor([[ 0.6585, -0.3228,  0.4086,  ..., -0.0924, -0.0929, -0.1885]],
       device='cuda:0', grad_fn=<CatBackward0>)

In [ ]:
# 1 layer
first_layer_emb = outputs['hidden_states'][1][0][token_index[0]]
print(first_layer_emb.shape)

# last layer
last_layer_emb = outputs['hidden_states'][-1][0][token_index[0]]
print(last_layer_emb.shape)

# sum all 12 layers
sum_all_layer_emb = torch.stack([outputs['hidden_states'][_][0][token_index[0]] for _ in range(len(outputs['hidden_states'][1:]))]).sum(axis=0)
print(sum_all_layer_emb.shape)

# sum last 4 layers
sum_last4_layer_emb = torch.stack([outputs['hidden_states'][_][0][token_index[0]] for _ in range(len(outputs['hidden_states'][-4:]))]).sum(axis=0)
print(sum_last4_layer_emb.shape)

# concat last 4 layers
concat_last4_layer_emb = torch.concat([outputs['hidden_states'][_][0][token_index[0]] for _ in range(len(outputs['hidden_states'][-4:]))], axis=1)
print(concat_last4_layer_emb.shape)

# mean last 4 layers
mean_last4_layer_emb = torch.stack([outputs['hidden_states'][_][0][token_index[0]] for _ in range(len(outputs['hidden_states'][-4:]))]).mean(axis=0)
print(mean_last4_layer_emb.shape)

torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 768])
torch.Size([1, 3072])
torch.Size([1, 768])


## Challenge

### Q3. `sum_last_four_layer` 방식으로 1번째 sequence의 2개의 "code" 토큰 사이의 코사인 유사도를 계산하라

$similarity=cos(Θ)=\frac{A⋅B}{||A||\ ||B||}=\frac{\sum_{i=1}^{n}{A_{i}×B_{i}}}{\sqrt{\sum_{i=1}^{n}(A_{i})^2}×\sqrt{\sum_{i=1}^{n}(B_{i})^2}}$

In [ ]:
def cosine_similarity_manual(x, y, small_number=1e-8):
    
    xnorm = torch.sqrt((x ** 2).sum())
    ynorm = torch.sqrt((y ** 2).sum())
    
    result = torch.mul(x, y).sum() / (xnorm * ynorm + small_number)

    return result

# input
# x: 1번째 sequence의 1번째 "code"의 sum_last_four_layer 방식 embedding
# y: 1번째 sequence의 2번째 "code"의 sum_last_four_layer 방식 embedding
x = torch.stack([outputs['hidden_states'][_][0][token_index[0]] for _ in range(len(outputs['hidden_states'][-4:]))]).sum(axis=0)
y = torch.stack([outputs['hidden_states'][_][0][token_index[1]] for _ in range(len(outputs['hidden_states'][-4:]))]).sum(axis=0)

# output
score = cosine_similarity_manual(x, y)
print(score)

tensor(0.9595, device='cuda:0', grad_fn=<DivBackward0>)


### Q4. 2번째 sequence에서 "coding"이라는 토큰의 위치를 반환하라

In [ ]:
# Q1과 동일한 문제 

# input
# seq1: 2번째 sequence
# token: 단어
seq2 = inputs['input_ids'][1]
token = "coding"

# output
# Q1에서 구현한 함수 사용
token_index = get_index(seq2, token)
print(token_index)

tensor([[10],
        [15]], device='cuda:0')


### Q5. `concat_last4_layer_emb` 방식으로 2번째 sequence의 2개의 "coding" 토큰 사이의 코사인 유사도를 계산하라

In [ ]:
# Q3과 동일한 문제

# input
# x: 2번째 sequence의 1번째 "coding"의 concat_last4_layer_emb
# y: 2번째 sequence의 2번째 "coding"의 concat_last4_layer_emb
x = torch.stack([outputs['hidden_states'][_][1][token_index[0]] for _ in range(len(outputs['hidden_states'][-4:]))]).view(1, -1)
y = torch.stack([outputs['hidden_states'][_][1][token_index[1]] for _ in range(len(outputs['hidden_states'][-4:]))]).view(1, -1)

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(score)

tensor(0.9159, device='cuda:0', grad_fn=<DivBackward0>)


### Q6. 2번째 sequence에서 랜덤하게 토큰 하나를 뽑아보자. 그 랜덤 토큰과 2번째 sequence의 2번째 "coding" 토큰의 코사인 유사도를 계산해보자

In [ ]:
# input
# random_idx: random 모듈 사용하여 뽑은 랜덤 토큰의 인덱스
# random_word: random_idx에 해당하는 단어
# x: 2번째 sequence의 2번째 "coding" 토큰의 concat_last4_layer_emb
# y: 랜덤 토큰의 concat_last4_layer_emb

random_idx = torch.randint(outputs['hidden_states'][0].shape[1], (1,))
random_word = tokenizer_bert.decode(inputs['input_ids'][1][random_idx])

x = torch.stack([outputs['hidden_states'][_][1][token_index[1]] for _ in range(len(outputs['hidden_states'][-4:]))]).view(1, -1)
y = torch.stack([outputs['hidden_states'][_][1][get_index(seq2, random_word)[0]] for _ in range(len(outputs['hidden_states'][-4:]))]).view(1, -1)

# output
# Q3에서 구현한 함수 사용
score = cosine_similarity_manual(x, y)
print(score)

tensor(0.1640, device='cuda:0', grad_fn=<DivBackward0>)


## Advanced

### Q7. 1번째 sequence와 2번째 sequence의 문장 유사도를 구해보자. 문장의 엠베딩은 마지막 레이어의 첫번째 토큰 ('[CLS]')으로 생성한다.

In [ ]:
# input
# x: 1번째 sequence의 embedding
# y: 2번째 sequence의 embedding
x = outputs['last_hidden_state'][0][0]
y = outputs['last_hidden_state'][1][0]

# output
# Q3에서 구현한 함수 사용
score =  cosine_similarity_manual(x, y)
print(score)

tensor(0.8130, device='cuda:0', grad_fn=<DivBackward0>)
